In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import boto3
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [2]:
path = 'processed_images'
paths = [os.path.join(path, f) for f in os.listdir(path)]

In [3]:
# Supondo que você tenha um detector de face (por exemplo, um classificador Haar)...
detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
def trainingModel(paths):
    faceSamples = []
    ids = []

    for imagePath in paths:
        if imagePath.endswith(('.png', '.jpg', '.jpeg')): # Filtra por formatos de imagem
            PIL_img = Image.open(imagePath).convert('L')
            img_uint8 = np.array(PIL_img, 'uint8')

            id = int(os.path.split(imagePath)[-1].split(".")[1])
            faces = detector.detectMultiScale(img_uint8)

            for (x, y, w, h) in faces:
                faceSamples.append(img_uint8[y:y+h, x:x+w])
                ids.append(id)

    return faceSamples, ids

In [5]:
faces, ids = trainingModel(paths)

In [19]:
x_train_faces, x_test_faces, y_train_ids, y_test_ids = train_test_split(faces, ids, test_size = 0.5, random_state = 42)

In [22]:
# Carregar o modelo treinado...
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [23]:
recognizer.train(x_train_faces, np.array(y_train_ids))
recognizer.write('trainer/trainer.yml')

In [ ]:
recognizer.read('trainer/trainer.yml')

In [25]:
# Realizar predições...
predicted_labels = []
for face in x_test_faces:
    label, confidence = recognizer.predict(face)
#     print('Confidence', confidence)
    predicted_labels.append(label)

# Converter listas para arrays do numpy...
test_labels = np.array(y_test_ids)
predicted_labels = np.array(predicted_labels)

KeyboardInterrupt: 

In [ ]:
purple_colors = ["#8e44ad", "#d7bde2", "#af7ac5", "#9b59b6", "#663399"]

accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
score = f1_score(test_labels, predicted_labels)

print("Precisão: ", precision)
print("Recall: ", recall)
print("F1 Score: ", score)

# Plotar a matriz de confusão
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap=sns.color_palette(purple_colors))
plt.ylabel('Rótulo Verdadeiro')
plt.xlabel('Rótulo Predito')
plt.title(f'Matriz de Confusão - Accuracy: {accuracy:.2f}')
plt.show()